## Input Reading

Each input data set is provided in a plain text file. The file contains only ASCII characters with lines ending with a single '\n' character (also called “UNIX-style” line endings). When multiple strings and numbers are given in one line, they are separated by a single space between each two elements.

The first line of the data set contains:

an integer C (1 ≤ C≤ 105) – the number of contributors,

an integer P (1 ≤ P ≤ 105) – the number of projects.

This is followed by C sections describing individual contributors. Each contributor is described by the following lines:

the first line contains:

the contributor's name (ASCII string of at most 20 characters, all of which are lowercase or uppercase English alphabet letters a-z and A-Z, or numbers 0-9),

an integer N (1≤ N ≤ 100) - the number of skills of the contributor.

the next N lines describe individual skills of the contributor. Each such line contains:

the name of the skill (ASCII string of at most 20 characters, all of which are lowercase or uppercase English alphabet letters a-z and A-Z, numbers 0-9, dashes '-' or pluses '+'),

an integer Li (1≤ Li ≤ 10) - skill level.

This is followed by P sections describing individual projects. Each project is described by the following lines:

the first line contains:
the name of the project (ASCII string of at most 20 characters, all of which are lowercase or uppercase English alphabet letters a-z and A-Z or numbers 0-9),

an integer Di (1 ≤Di ≤ 105) – the number of days it takes to complete the project,

an integer Si (1 ≤ Si ≤ 105) – the score awarded for project’s completion,

an integer Bi (1 ≤ Bi ≤ 105) – the “best before” day for the project,

an integer Ri (1 ≤ Ri ≤ 100) – the number of roles in the project.

the next Ri lines describe the skills in the project:

a string Xk – the name of the skill (ASCII string of at most 20 characters, all of which are lowercase or uppercase English alphabet letters a-z and A-Z, numbers 0-9, dashes '-' or pluses '+'),

an integer Lk (1≤Lk≤100) – the required skill level.

In [14]:
# Variable/function Declaration
contributors = []
projects = []
file = 'b'

# Initialise first line of values in input
text = open(file+'.txt', 'r').readlines()
firstline = text[0]
firstline = firstline.split(sep=None)
contributor_no = int(firstline[0])
project_no = int(firstline[1])

# Initialise list of contributors
linejump = 1
for x in range(contributor_no):
    # Read the line describing the contributor
    line = text[x+linejump]
    line = line.split(sep=None)
    
    # Read the details of the contributor
    name = line[0]
    skills = int(line[1])
    contributor = {}
    contributor['name'] = name
    for y in range(skills):
        linejump += 1
        skill = text[x+linejump]
        skill = skill.split(sep=None)
        contributor[skill[0]] = int(skill[1])
    # Add contributor data to the list of contributors
    contributors.append(contributor)
linejump += contributor_no # Now it points to the first project in the whole project section

# Initialise list of projects
for x in range(project_no):
    # Read the line describing the project
    line = text[x+linejump]
    line = line.split(sep=None)
    
    # Read the details of the project
    name = line[0]
    duration = int(line[1])
    score = int(line[2])
    deadline = int(line[3])
    role_no = int(line[4])
    project = {}
    project['name'] = name
    project['duration'] = duration
    project['score'] = score
    project['deadline'] = deadline
    roles = []
    for y in range(role_no):
        linejump += 1
        role = text[x+linejump]
        role = role.split(sep=None)
        role = (role[0], int(role[1]))
        roles.append(role)
    project['roles'] = roles
    projects.append(project)

## Functions

In [9]:
import operator

def getRoleDistribution(solution, pname):
    for s in solution:
        if (s.get("name") == pname):
            return s;
def getThing(solution, pname):
    for i, s in enumerate(solution):
        if (s.get("name") == pname):
            return i;


def find_contributors(contributors,proj): #whole contributors, but single project
    roles=proj["roles"]
    contributorPotentialList=[]
    contributorDict=[]
    solnList=[1 for x in range(len(proj["roles"]))]
    
    for contributor in contributors:
        subDict={}
        subDict["name"]=contributor["name"]
        subDict["count"]=0
        subDict["index_list"]=[]
        contributorDict.append(subDict)
    
    #Iterate through every roles, add the potential candidates into the contributorPotentialList
    ind=0
    for i in range(0,len(roles)):
        skill=str(roles[i][0])
        skillLevelRequired=roles[i][1]
        
        contributorPotentialSublist=[]
        for contributor in contributors:
            try:
                contributorSkillLevel=contributor[skill]
                if (contributorSkillLevel>=skillLevelRequired):
                    contributorPotentialSublist.append(contributor["name"])
                    
                    cName=contributor["name"]
                    for c in contributorDict:
                        if c["name"]==cName:
                            c["count"]=c["count"]+1
                            c["index_list"].append(ind)
            except:
                continue
        contributorPotentialList.append(contributorPotentialSublist)
        ind+=1
    
#     for x in range(len(contributorDict)):
#         contributorDict[x]['index_list'].sort()

#     print(contributorPotentialList,'\n', contributorDict,'\n\n')


    for j in range(len(proj["roles"])):
        contributorDict=sorted(contributorDict,key=operator.itemgetter('count'))
        x=0
        while x<len(contributorDict):
#             print(contributorPotentialList,'\n', contributorDict,'\n\n')
            if contributorDict[x]['count']==0:
                contributorDict.pop(x)
#                 del contributorDict[x]
#                 print(contributorPotentialList,'\n', contributorDict,'\n\n')
            else:
                break
#         print(contributorPotentialList,'\n', contributorDict,'\n\n')
        skillIndex=contributorDict[0]["index_list"][0]
        name=contributorDict[0]["name"]
        r=getRoleDistribution(contributors,name)
        solnList[skillIndex]=r
#         contributorDict.remove(r)
        o=getThing(contributorDict, name)
        contributorDict[o]["count"]=0
        contributorDict[o]["index_list"]=[]
        for i in range(len(contributorDict)):
            try:
                contributorDict[i]["index_list"].remove(skillIndex)
                contributorDict[i]["count"]-=1
            except:
                continue
        
#     print(solnList)
    return solnList

# level up contributors in place, project is just project, contributors is a list of contributor working on the project
# for now assume that no mentoring happens
def upgrade(project, contributors):
    tasks = project['roles']
    for x in range(len(tasks)):
        if tasks[x][1] >= contributors[x][tasks[x][0]]:
            contributors[x][tasks[x][0]] += 1
    return contributors

def output(projects, contributors):
#     projects_len=len(projects)
#     with open("output.txt", "a") as a:
#         a.write(str(projects_len)+"\n")
    
#     for i in range(0,projects_len):
    psol = []
    csol = []
    
    for proj in projects:
        try:
            output=find_contributors(contributors,proj)
            if proj["name"] == "Logging":
                print(output)
        except:
            continue
        projects.remove(proj)
        upgrade(proj, output)
        for q in output:
            contributors[getThing(contributors, q["name"])] = q
        psol.append(proj)
        tempar = []
        for i in output:
            tempar.append(i["name"])
        csol.append(tempar)
        
    for proj in projects:
        try:
            output=find_contributors(contributors,proj)
            if proj["name"] == "Logging":
                print(output)
        except:
            continue
        upgrade(proj, output)
        
        for q in output:
            contributors[getThing(contributors, q["name"])] = q 

        psol.append(proj)
        tempar = []
        for i in output:
            tempar.append(i["name"])
        csol.append(tempar)
        
    return psol, csol

### Testing

## Input printer

Your submission describes which projects each contributor is going to work on and in which role.

The first line should contain the integer E (0≤E≤P) – the number of executed projects.

This should be followed by E sections each describing one completed project. Each project should be described by two lines:

A single line containing the name of the project (as it appears in the input file). Each project can be mentioned at most once in the submission file.

A single line containing the names of the contributors assigned to each of the project roles, separated by single spaces and listed in the same order as the roles appear in the input file.

In [3]:
# p: list of project dictionaries that are done
# contributors: list of list of contributor names

def writeSol(p, contributors, fname):
    with open("solution"+fname[0]+".txt", "w") as w:
        w.write(str(len(p))+"\n")
        for (i, proj) in enumerate(p):
            w.write(proj["name"]+"\n")
            for j in contributors[i]:
                w.write(j+" ")
            w.write("\n")

In [15]:
psol, csol = output(projects, contributors)
writeSol(psol, csol, file)